# MODULE 1: MAKING SENSE OF UNSTRUCTURED DATA

## Case Study - Data Analysis with Human-Generated Text

In this document, we walk through some tips to help you with doing your own analysis on MIT EECS 
faculty data using stochastic variational inference on LDA.

1. Scraping your own dataset
2. Pre-processing the dataset
3. Implementing your own LDA code


**Implementing your own SVI-LDA code** 

Latent Dirichlet allocation (LDA) is a generative statistical model in natural language processing, and 
can be used to discover ‘topics’ in a large set of documents. This is first presented by David Blei, 
Andrew Ng, and Michael Jordan. 

The key idea is that if we see a ‘topic’ as a collection of certain 
words, we can look at each document as a collection of topics, the proportion of each topic depends 
on the proportion of words in the document that are associated with that topic. For example, the 
‘sports’ topic may consist of the words: tennis, football, gymnastics.
When given a set of documents, we can calculate the posterior distribution for the topics. In the 
original LDA paper, this is done using a coordinate descent algorithm for mean-field variational
inference, and later on researchers also used Gibbs Sampling and expectation propagation.
In this tutorial we will be looking only at Stochastic Variational Inference for LDA. SVI was first 
published in 2013 by Matt Hoffman, David Blei, Chong Wang, and John Paisley.

Traditional coordinate-descent variational inference requires each update to be carried out with all of the data, 
and these updates become inefficient when the dataset gets large as each update scales linearly 
with the size of the data. The key idea with SVI is to update global variational parameters more 
frequently.
Using local and global parameters, and given the dataset with a known number of datapoints, we 
could randomly take 1 data point at a time, update the local parameter, and project the change into 
the global parameters. Like traditional coordinate-descent variational inference, this is done until the 
result converges, i.e., the change in the global parameters is smaller than a certain value.
The implementation we will be talking about is a naive implementation of the algorithm described in 
the original paper

.
**Variable Notation**

Here we provide a brief overview of the input variables for LDA and SVI. Variables that can be set are 
the following: 

• λ: what we want in the end (the posterior distribution for the topics for each word

• vocab: this is the overall vocabulary we will have in the docs

• K: this is the number of topics we want to get in the end

• D: this is the total number of documents

• α: parameter for per-document topic distribution

• η: parameter for per-topic vocab distribution2017 © Massachusetts Institute of Technology

• τ: delay that down weights early iterations

• κ: forgetting rate, controls how quickly old information is forgotten; the larger the value, the 
slower it is.

• max:iterations: the number of maximum iterations the updates should go on for. We usually 
set a check such that if the difference in two consecutive values of λ is smaller than a certain 
value, we say the algorithm has converged. However, sometimes we could set this certain 
value too small, so we set a maximum iteration value to avoid updates running forever.

**LDA Generative Model**

We review the LDA generative model here. LDA assumes each document has K topics with different 
proportions. It models a corpus w of size D as follows:
    
• Draw distribution over vocabulary βk ~ Dirichlet(η) for topics k ∈ {1…K}

• For each document d ∈ {1…D} :
    
– Draw topic proportions θd ~ Dirichlet(α);

– For each word 𝑊𝑑 𝑛 in the document:
    
* Draw topic indicator 𝑍𝑑 𝑛~ Multinomial (θd)

* Draw word 𝑊𝑑 𝑛 ~ Multinomial (β𝑍𝑑𝑛)

Note that this model follows the ‘bag of words’ assumption, such that given the topic proportions, 
each word drawn is independent of any other words in the document.

<h2> Libraries </h2>

In [1]:
from bs4 import BeautifulSoup
from requests import get
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import collections
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luca.olivini\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luca.olivini\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# LDA Generative Model

Priors: 
- Distribution over vocabulary for topic k in {1..K}: beta\[k\] ~ Dirichlet(V, eta)
- Distribution over topics (latent variables): theta ~ Dirichlet(K, alpha)

For each document:
- Choose number of words: N ~ Poisson(ξ)

For each of the N words w:
- Choose a topic: z ~ Cat(K, theta)
- Choose a word: w ~ Cat(V, beta\[z\])

Note: This model follows the ‘bag of words’ assumption, such that given the topic proportions,
each word drawn is independent of any other words in the document. 

![Graphical representation](images/LDA_PGM_representation.PNG)

### Variational Inference

To use variational inference, the edges between θ (theta), z and w are removed to make inference on LDA model tractable. 

![Variational Inference](images\Variational_Distribution_representation.PNG)

# Global Variables

In [2]:
faculty_url = 'https://www.eecs.mit.edu/role/faculty/?fwp_role=faculty'
arXiv_format = 'arxiv.org/find/{}/1/au:+{}_{}/0/1/0/all/0/1' # arxiv.org/find/(subject)/1/au:+(lastname)_(initial)/0/1/0/all/0/1
search_url_format = 'https://arxiv.org/search/?query="{}"&searchtype=author'
subjects = {'Computer Science': 'Computer Science', 
            'Electrical Engineering': 'Electrical Engineering and Systems Science',
            'Physics': 'Physics'}
all_papers_columns = ['Name', 'Abstract']

# Web Sraping

1. Get Facultys

Using BeautifulSoup (https://www.crummy.com/software/BeautifulSoup/), and by analyzing the 
structure of the source code of arXiv, we could scrape the name list of MIT EECS faculty members. 
Using this information, we could list the query we send to arXiv. 
A possible format for the arXiv search for papers by authors is the following:

arxiv.org/find/(subject)/1/au:+(lastname)_(initial)/0/1/0/all/0/1

You could therefore adapt the names you scraped, and query through all the relevant arXiv search 
pages.

Within the arXiv source code, look for < class span=list-identifier >, which will give the identifier for 
the papers listed in your query results. Similarly look for the tag for the “Abstract” within each paper 
and scrape the abstract for each paper you find.

Note that you might want to scrape more information than you need and then do some local 
processing with the text you have instead.


In [3]:
from urllib.request import Request, urlopen
faculty_url = "https://www.eecs.mit.edu/role/faculty/?fwp_role=faculty"
hdr = {"User-Agent":"Mozilla/5.0"}
req = Request(faculty_url, headers=hdr)
page = urlopen(req)
faculty_page_content = BeautifulSoup(page, 'html.parser')

In [4]:
names = []
names = [x.text for x in faculty_page_content.find_all('h5')]
print(names)

['Hal Abelson', 'Elfar Adalsteinsson', 'Fadel Adib', 'Anant Agarwal', 'Pulkit Agrawal', 'Akintunde Akinwande', 'Mohammad Alizadeh', 'Saman Amarasinghe', 'Jacob Andreas', 'Dimitri Antoniadis', 'Arvind', 'Ahmad Bahai', 'Hari Balakrishnan', 'Marc A Baldo', 'Regina Barzilay', 'Adam Belay', 'Karl Berggren', 'Dimitri Bertsekas', 'Robert Berwick', 'Sangeeta Bhatia', 'Duane Boning', 'Guy Bresler', 'Tamara Broderick', 'Vladimir Bulović', 'Michael Carbin', 'Vincent Chan', 'Anantha Chandrakasan', 'YuFeng (Kevin)  Chen', 'Adam Chlipala', 'Isaac Chuang', 'Connor Wilson Coley', 'Henry Corrigan-Gibbs', 'Munther Dahleh', 'Luca Daniel', 'Constantinos Daskalakis', 'Randall Davis', 'Jesús del Alamo', 'Erik Demaine', 'Srini Devadas', 'David DeWitt', 'Frederic Durand', 'Joel Emer', 'Dirk Englund', 'Dennis Freeman', 'William Freeman', 'James Fujimoto', 'Mohsen Ghaffari', 'Marzyeh Ghassemi', 'Manya Ghobadi', 'David Gifford', 'Shafrira Goldwasser', 'Polina Golland', 'Martha Gray', 'W. Eric L Grimson', 'John G

2. Scrape Papers

In [25]:
#define a function in order to scrap the papers 
def scrapeArXiv(names):
    papers = list()
    for name in names:
        search_url = search_url_format.format(name.replace(' ','+'))
        papers_author = get(search_url)
        papers_author_content = BeautifulSoup(papers_author.content, 'html.parser')
        papers_author_body = papers_author_content.body
        results = papers_author_body.find_all("li", class_ = 'arxiv-result')
        abstracts = [result.find('span', class_ ='abstract-full') for result in results]
        
        abstracts_content = [abstract.a.unwrap() for abstract in abstracts]
        abstracts_content = [abstract.contents[0] for abstract in abstracts]
        
        if abstracts_content : 
            papers = papers + abstracts_content
        
    return papers

In [26]:
papers = scrapeArXiv(names)

In [21]:
papers

["\n        We investigate whether three types of post hoc model explanations--feature attribution, concept activation, and training point ranking--are effective for detecting a model's reliance on spurious signals in the training data. Specifically, we consider the scenario where the spurious signal to be detected is unknown, at test-time, to the user of the explanation method. We design an empirical methodology that uses semi-synthetic datasets along with pre-specified spurious artifacts to obtain models that verifiably rely on these spurious training signals. We then provide a suite of metrics that assess an explanation method's reliability for spurious signal detection under various conditions. We find that the post hoc explanation methods tested are ineffective when the spurious artifact is unknown at test-time especially for non-visible artifacts like a background blur. Further, we find that feature attribution methods are susceptible to erroneously indicating dependence on spuri

# Text Preprocessing

Pre-processing the dataset

In the original work we have processed the data as raw documents as the dataset size was small. 
However if you want to use Matthew Hoffman’s original SVI code instead, that code takes a text 
file with a specific format. Once you have each abstract in a separate text file, you may find the 2017 © Massachusetts Institute of Technology  following Python packages useful: io, collections, nltk. It is good practice to keep your dataset in its own folder, so io can be used to access that folder using a constant (relative) path. Read each file 
and use nltk.tokenize to tokenize each chunk of text. Use collections to process each abstract using 
a Counter/Dictionary, before writing the counts of words of each individual abstract as a line in the 
text file.

In [8]:
# define a function in order to cleaning and counts the word of each paper 
def word_cleaning_and_count(s):
    s_lower = s.lower()
    cleaning_set = set(stopwords.words('english')) # this is a set of all the word that in english is connector like the, than ecc
    tokens = word_tokenize(s_lower)
    tokens = [token for token in tokens if token.isalpha()] # isalpha() usefull because it find the puntuction
    word_dict = dict(collections.Counter(tokens)) # we create a dict where we have the:8, n:25 etc
    for key in cleaning_set:
        word_dict.pop(key,None) # None beacause if the word in the set: cleaning_set is not present in the word_dict there isn't error 
    return word_dict

In [9]:
papers_word_dict = [word_cleaning_and_count(paper) for paper in papers]
dup_keys = []
for i in range(len(papers_word_dict)):
    dup_keys = dup_keys + list(papers_word_dict[i].keys()) #we create a list of all the possible keys

vocab = list(collections.Counter(dup_keys).keys()) # we count how many time a single word is present across all the possible article
lookup_table = dict(zip(vocab, range(len(vocab)))) # we create a dictionary in which we have a sort of words dictionary in which we associate to all the words a position

<b> Save data </b>

In [10]:
import json
with open('data/names','w') as fout:
    json.dump(names,fout)
with open('data/papers','w') as fout:
    json.dump(papers,fout)
with open('data/papers_word_dict','w') as fout:
    json.dump(papers_word_dict,fout)
with open('data/vocab','w') as fout:
    json.dump(vocab, fout)
with open('data/lookup_table', 'w') as fout:
    json.dump(lookup_table,fout)

<h2> LDA </h2>

In [11]:
#similar to k in K-means clustering. We want to divide abstracts into 5 topics.
no_topics = 5

<b> Load Data </b>

In [12]:
import json
with open('data/names', 'r') as json_file:
    names = json.load(json_file)
with open('data/papers','r') as json_file:
    papers = json.load(json_file)
with open('data/papers_word_dict', 'r') as json_file:
    papers_word_dict = json.load(json_file)
with open('data/vocab', 'r') as json_file:
    vocab = json.load(json_file)
with open('data/lookup_table', 'r') as json_file:
    lookup_table = json.load(json_file)

vocab_size = len(vocab)

<b> Using sklearn </b>

In [18]:
doc_vecs = []
for paper in papers_word_dict:
    doc_vec = [0 for _ in range(vocab_size)]
    for token,occurs in paper.items():
        doc_vec[lookup_table[token]] = occurs
    doc_vecs.append(doc_vec)

In [19]:
from sklearn.decomposition import LatentDirichletAllocation

# Run the LDA 
lda = LatentDirichletAllocation(n_components=no_topics, learning_method='online').fit(doc_vecs)

# define a function for displaying the result
def display_topics(model, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" %(topic_idx))
        print(" ".join([vocab[i] for i in topic.argsort()[:-no_top_words -1 : -1]]))
        
# using top 10 words present in each topic
no_top_words = 10
display_topics(lda, no_top_words)

Topic 0:
image data images clinical motion segmentation mri imaging brain system
Topic 1:
data learning model models performance show neural methods training approach
Topic 2:
quantum energy systems optical devices using materials magnetic system properties
Topic 3:
objects robot planning object gradient functions function scene control manipulation
Topic 4:
n algorithm algorithms time problem graph show k also lower


<b> End-to-end Code (SVILDA algorithm) </b>

In [15]:
doc_vecs = []
for paper in papers_word_dict:
    wordlist = []
    countlist = []
    for token, occurs in paper.items():
        wordlist.append(lookup_table[token])
        countlist.append(occurs)
    doc_vecs.append((wordlist, countlist))


In [16]:
from svilda import SVILDA
iterations = 10000
lda = SVILDA(vocab, no_topics, len(doc_vecs), 0.1, 0.01, 1, 0.75, iterations)
lda.runSVI(doc_vecs)

ITERATION 0  running document number  368
ITERATION 100  running document number  2089
ITERATION 200  running document number  1309
ITERATION 300  running document number  3014
ITERATION 400  running document number  1637
ITERATION 500  running document number  3067
ITERATION 600  running document number  1658
ITERATION 700  running document number  1929
ITERATION 800  running document number  2452
ITERATION 900  running document number  2239
ITERATION 1000  running document number  2501
ITERATION 1100  running document number  1280
ITERATION 1200  running document number  2916
ITERATION 1300  running document number  84
ITERATION 1400  running document number  1120
ITERATION 1500  running document number  872
ITERATION 1600  running document number  589
ITERATION 1700  running document number  1323
ITERATION 1800  running document number  2830
ITERATION 1900  running document number  2273
ITERATION 2000  running document number  2872
ITERATION 2100  running document number  1062
ITERA

In [17]:
def display_topics(model, no_top_words):
    for topic_idx, topic in enumerate(model._lambda):
        print('Topic %d:' % (topic_idx))
        print(' '.join([vocab[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
no_top_words = 10
display_topics(lda,no_top_words)

Topic 0:
data models using demonstrate one use set large applications existing
Topic 1:
show work performance graph problems system different study framework k
Topic 2:
problem paper results new time first systems quantum provide used
Topic 3:
learning n also methods training network two neural propose present
Topic 4:
model algorithm algorithms method networks approach based number tasks optimization


<b> Conclusion </b>

- SVILDA gives better result as compared to LDA. 
- SVILDA is able to group topics more precisely into different and meaningful clusters.
- The research papers are mostly related to algorithms, core computer science and machine/deep learning.
- Other than computer science the topics are related to Quantum theory and material science. 